Test 3 VGG16

In [1]:
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

Loading data into numpy array


Below is the code for loading the image data from the train, test, and validation folders into numpy arrays, and performing normalization of every pixel in the range of 0 and 1

In [2]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

In [3]:
# Example usage:
data_dir = "F:/thesis/data"

In [4]:
def load_data(dataset_dir):
    images = []
    labels = []

    # Get the total number of images
    num_images = sum(len(files) for _, _, files in os.walk(dataset_dir))

    # Create tqdm progress bar
    pbar = tqdm(total=num_images, desc=f'Loading {dataset_dir}', unit='image')

    # Iterate through each subfolder in the dataset directory
    for class_folder in sorted(os.listdir(dataset_dir)):
        class_dir = os.path.join(dataset_dir, class_folder)
        if os.path.isdir(class_dir):
            # Iterate through each image file in the class folder
            for image_file in sorted(os.listdir(class_dir)):
                image_path = os.path.join(class_dir, image_file)
                # Load image using PIL
                image = Image.open(image_path)
                # Resize image to 224x224 if necessary (optional)
                image = image.resize((224, 224))
                # Convert image to numpy array and normalize pixel values
                image = np.array(image) / 255.0
                # Append image and corresponding label to lists
                images.append(image)
                labels.append(int(class_folder))
                # Update progress bar
                pbar.update(1)

    # Close progress bar after completion
    pbar.close()

    return np.array(images), np.array(labels)

# Load data for training set
train_images, train_labels = load_data(os.path.join(data_dir, 'train'))

# Load data for test set
test_images, test_labels = load_data(os.path.join(data_dir, 'test'))

# Load data for validation set
val_images, val_labels = load_data(os.path.join(data_dir, 'validation'))

Loading F:/thesis/data\train:   0%|          | 0/5180 [00:00<?, ?image/s]

Loading F:/thesis/data\train:  36%|███▌      | 1862/5180 [00:04<00:10, 310.11image/s]

In [ ]:
print("Shape of train images array:", train_images.shape)
print("Shape of train labels array:", train_labels.shape)
print("Shape of test images array:", test_images.shape)
print("Shape of test labels array:", test_labels.shape)
print("Shape of validation images array:", val_images.shape)
print("Shape of validation labels array:", val_labels.shape)
print(train_images)
print(type(train_images))

Shape of train images array: (5180, 224, 224, 3)
Shape of train labels array: (5180,)
Shape of test images array: (1748, 224, 224, 3)
Shape of test labels array: (1748,)
Shape of validation images array: (1742, 224, 224, 3)
Shape of validation labels array: (1742,)
[[[[0.63137255 0.58431373 0.49803922]
   [0.62745098 0.58039216 0.49411765]
   [0.63529412 0.58823529 0.50196078]
   ...
   [0.59607843 0.53333333 0.4745098 ]
   [0.57647059 0.51372549 0.45490196]
   [0.6        0.5372549  0.47843137]]

  [[0.63137255 0.58431373 0.49803922]
   [0.63529412 0.58823529 0.50196078]
   [0.64705882 0.6        0.51372549]
   ...
   [0.58431373 0.52156863 0.4627451 ]
   [0.59215686 0.52941176 0.47058824]
   [0.57254902 0.50980392 0.45098039]]

  [[0.63529412 0.58823529 0.50196078]
   [0.64313725 0.59607843 0.50980392]
   [0.65882353 0.61176471 0.5254902 ]
   ...
   [0.61176471 0.54901961 0.49019608]
   [0.60784314 0.54509804 0.48627451]
   [0.58823529 0.5254902  0.46666667]]

  ...

  [[0.80784314 0

Model

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
# Number of classes (38 in your case)
num_classes = 38

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model with the custom classification head
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


In [ ]:
# Assuming you have already loaded your data into variables train_images, train_labels, test_images, test_labels, val_images, val_labels

# Train the model
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    batch_size=32,
                    epochs=10,
                    verbose=1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")